# Tracking an object
This is a follow up of previous two tutorials. If you haven't studied them before, it is recommended to study them first

## In this lesson, we will learn:
* to calculate euclidean distance
* drawing on image and video
* simple trignometry (sine and cosine law)